In [ ]:
%%capture
!pip install openai langchain

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') #Make sure to include your openAI token in the secret code section in colab anc activate it

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.schema import Document

class ConversationalAgentLLM:
    def __init__(self):
        self.user_profile = {}
        self.questions = [
            ("plan_type", "What kind of plan are you working on? (e.g., vacation, business trip, event, project)"),
            ("plan_description", "Could you provide a brief description of what you're planning?"),
            ("dates", "Do you have specific dates in mind for this? If so, when will it start and end?"),
            ("date_flexibility", "Are your dates flexible, or are they fixed?"),
            ("location", "Where will this take place? If it’s a trip, what are your destination(s)?"),
            ("accommodations", "Do you need help with accommodations, venues, or specific locations?"),
            ("budget", "Do you have a budget in mind for this plan?"),
            ("budget_help", "Would you like assistance in finding cost-effective options within your budget?"),
            ("preferences", "Do you have any specific preferences or requirements? (e.g., dietary restrictions, preferred airlines, venue amenities)"),
            ("activities", "Are there any particular activities or services you’re interested in?"),
            ("people_count", "How many people will be involved in this plan?"),
            ("special_considerations", "Are there any special considerations for the group? (e.g., children, elderly, special needs)"),
            ("past_experience", "Have you done something similar before? If yes, what was your experience like?"),
            ("repeat_or_avoid", "Is there anything from your past experiences that you'd like to repeat or avoid?"),
            ("additional_info", "Is there anything else you’d like to share about your plans?")
        ]
        self.current_question_index = 0
        self.conversation_ended = False
        self.chain = self.initialize_chain()

    def initialize_chain(self):
        return load_qa_chain(OpenAI(), chain_type="stuff", prompt=self.get_prompt_template())

    def get_prompt_template(self):
        template = """
        Answer the question based on the user profile below. Do your best to provide a helpful response or suggest additional information that might be needed.

        Context: {context}

        Question: {question}

        Answer: """
        return PromptTemplate(template=template, input_variables=['context', 'question'])

    def ask_next_question(self):
        if self.current_question_index < len(self.questions):
            key, question = self.questions[self.current_question_index]
            self.current_question_index += 1
            return key, question
        else:
            return None, "Now that I have collected all the initial information, how may I assist you further? Feel free to ask me anything."

    def store_response(self, key, response):
        self.user_profile[key] = response

    def build_context(self):
        return "\n".join([f"{k}: {v}" for k, v in self.user_profile.items()])

    def run(self):
        self.conversation_ended = False
        self.output_area = widgets.Output()
        self.response_area = widgets.Textarea(
            value='',
            placeholder='Model responses will appear here...',
            description='Response:',
            disabled=True,
            layout=widgets.Layout(width='80%', height='200px')
        )
        display(self.output_area, self.response_area)
        self.collect_initial_profile()

    def collect_initial_profile(self):
        self.display_message("Hello! I’m here to help you with your plans. Could you tell me a bit more about what you're planning? Whether it’s a trip, event, or project, I’m here to assist!")
        self.ask_profile_question()

    def ask_profile_question(self):
        if self.current_question_index < len(self.questions):
            key, question = self.ask_next_question()
            self.display_question(question)
        else:
            self.display_message("Now that I have collected all the initial information, how may I assist you further? Feel free to ask me anything.")
            self.create_input_widgets()

    def display_message(self, message):
        with self.output_area:
            clear_output(wait=True)
            print(message)

    def display_question(self, question):
        self.input_widget = widgets.Text(placeholder=question, layout=widgets.Layout(width='80%'))
        self.input_widget.on_submit(self.handle_profile_response)
        with self.output_area:
            clear_output(wait=True)
            display(self.input_widget)

    def handle_profile_response(self, text_widget):
        response = text_widget.value
        key, question = self.questions[self.current_question_index - 1]
        self.store_response(key, response)
        self.ask_profile_question()

    def create_input_widgets(self):
        self.user_input = widgets.Text(placeholder="Enter your question here", layout=widgets.Layout(width='80%'))
        self.user_input.on_submit(self.handle_user_question)
        self.stop_button = widgets.Button(description="Bye", layout=widgets.Layout(width='10%'))
        self.stop_button.on_click(self.end_conversation)
        with self.output_area:
            clear_output(wait=True)
            display(self.user_input, self.stop_button)

    def handle_user_question(self, text_widget):
        user_question = text_widget.value
        if user_question.lower() in ["exit", "quit", "stop"]:
            self.end_conversation(None)
        else:
            context = self.build_context()
            document_context = Document(page_content=context)
            response = self.chain.run(input_documents=[document_context], question=user_question)
            self.store_response("last_user_question", user_question)
            self.store_response("last_response", response)
            self.response_area.value = response
            self.create_input_widgets()

    def end_conversation(self, button):
        self.conversation_ended = True
        self.display_message("Thank you for using our service. Have a great day!")

    def extract_and_store_additional_info(self, response):
        print("Updating profile with new information from the response...")


In [ ]:
#@title Usage
# You first answer priliminary questions to enrich your profil before starting a personalised conversation with the LLM, here ChatGPT.
agent = ConversationalAgentLLM()
agent.run()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Output()

Textarea(value='', description='Response:', disabled=True, layout=Layout(height='200px', width='80%'), placeho…

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
